In [1]:
### COMTRADE DATASETS EXTRACTING

In [2]:
### RUN EVERY TIME: INITIALIZATION

import pandas as pd
import numpy as np
import requests
import gc
import os
import datetime
import time

In [3]:
### RUN EVERY TIME: VERSION CONTROL

from platform import python_version
print('python version: ', python_version())
print('numpy version: ', np.__version__)
print('pandas version: ', pd.__version__)

python version:  3.7.4
numpy version:  1.17.2
pandas version:  0.25.3


In [4]:
### RUN EVERY TIME: MAIN CONSTANTS

### MultiIndex level slice constant:
All = slice(None)
### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### Dates:
str_date_end = '2022-12-31'
date_start = pd.Timestamp('1989-12-29')
date_end = pd.Timestamp(str_date_end)
### NA for MS Excel files:
list_na_excel_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null',
                        '#N/A Requesting Data...', '#N/A Invalid Security', '#N/A Field Not Applicable', '---']
### UN Comtrade adopted data containers:
str_path_unc_res_all_annual = 'Data_Files/Source_Files/unc_res_all_annual.h5'
str_key_unc_res = 'unc_res'
### File with aggregated flows:
str_path_unc_res_flows = 'Data_Files/Source_Files/unc_res_flows.h5'
### Universal HDF5 key:
str_key_unc_res = 'unc_res'
### Augmented bilateral export:
str_path_export_bilateral = 'Data_Files/Source_Files/comtrade_export_bilateral.h5'
### Export key:
str_key_unc_export = 'export_augmented'
### Augmented bilateral import:
str_path_import_bilateral = 'Data_Files/Source_Files/comtrade_import_bilateral.h5'
### Import key:
str_key_unc_import = 'import_augmented'

In [5]:
### DEFINING COUNTRY CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    ### In case if URL is unavailable:
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    ### Online extraction:
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']].sort_index()    
    df_result.index = df_result.index.str.upper()
    ### Results output:
    return df_result

In [6]:
### UN COMTRADE: DATA EXTRACTION

In [8]:
### UN COMTRADE: CODELISTS LOADING (PRODUCT VERSION)

### World Country Codes:
df_country_codes = get_country_codes()
### Codelists container:
dict_codelist = {}
### List of reference tables for categories:
request_session = requests.Session()
obj_unc_reference = request_session.get('https://comtradeapi.un.org/files/v1/app/reference/ListofReferences.json')
df_cat_reference = pd.DataFrame(obj_unc_reference.json()['results']).set_index('category')
### Description of categories
df_cat_description = pd.DataFrame(request_session.get(df_cat_reference.loc['dataitem']['fileuri']).json()['results']).set_index('dataItem')
### Parameters:
ser_type_code = pd.Series(['Goods', 'Services'], index = ['C', 'S'])
ser_type_code.index.names = ['id']
ser_type_code.name = 'typeCode'
dict_codelist['typeCode'] = ser_type_code
### Frequency:
ser_freq_code = pd.DataFrame(request_session.get(df_cat_reference.loc['freq']['fileuri']).json()['results']).set_index('id').squeeze()
dict_codelist['freqCode'] = ser_freq_code
### Commodity HS Codes:
df_hs_comm = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:HS']['fileuri']).json()['results'])
ser_hs_comm_ag2 = df_hs_comm[df_hs_comm['aggrLevel'] == 2].drop(['parent', 'isLeaf', 'aggrLevel'], axis = 1).set_index('id').squeeze().str[5: ]
ser_hs_comm_ag2.name = 'clCode'
dict_codelist['clCode'] = {}
dict_codelist['clCode']['C'] = ser_hs_comm_ag2
### Service EBOPS Codes:
df_eb_serv = pd.DataFrame(request_session.get(df_cat_reference.loc['cmd:EB']['fileuri']).json()['results'])
df_eb_serv_ag2 = df_eb_serv[df_eb_serv['parent'].isin(df_eb_serv[df_eb_serv['parent'] == '200']['id'])]
ser_eb_serv_ag2 = df_eb_serv_ag2[df_eb_serv_ag2['id'].astype(int) <= 950].set_index('id')['text']
ser_eb_serv_ag2.name = 'clCode'
dict_codelist['clCode']['S'] = ser_eb_serv_ag2
### United codes:
dict_codelist['clCode']['T'] = pd.concat([dict_codelist['clCode']['C'], dict_codelist['clCode']['S']], axis = 0)
### Reporter Codes:
df_reporter_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['reporter']['fileuri']).json()['results'])
df_reporter_raw['id'] = df_reporter_raw['id'].astype(str).str.zfill(3)
df_reporter_raw['entryEffectiveDate'] = pd.to_datetime(df_reporter_raw['entryEffectiveDate'])
df_reporter_raw['entryExpiredDate'] = pd.to_datetime(df_reporter_raw['entryExpiredDate'])
df_reporter_raw = df_reporter_raw[df_reporter_raw['reporterCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_reporter_raw = df_reporter_raw[df_reporter_raw['entryExpiredDate'].isna() | (df_reporter_raw['entryExpiredDate'] > date_start)]
ser_reporter_code = df_reporter_raw.set_index('id')['reporterCodeIsoAlpha2'].squeeze()
ser_reporter_code.name = 'Reporter'
dict_codelist['reporterCode'] = ser_reporter_code
### Partner Codes:
df_partner_raw = pd.DataFrame(request_session.get(df_cat_reference.loc['partner']['fileuri']).json()['results'])
df_partner_raw['id'] = df_partner_raw['id'].astype(str).str.zfill(3)
df_partner_raw['entryEffectiveDate'] = pd.to_datetime(df_partner_raw['entryEffectiveDate'])
df_partner_raw['entryExpiredDate'] = pd.to_datetime(df_partner_raw['entryExpiredDate'])
df_partner_raw = df_partner_raw[df_partner_raw['PartnerCodeIsoAlpha2'].isin(df_country_codes['ISO SHORT'])]
df_partner_raw = df_partner_raw[df_partner_raw['entryExpiredDate'].isna() | (df_partner_raw['entryExpiredDate'] > date_start)]
ser_partner_code = df_partner_raw.set_index('id')['PartnerCodeIsoAlpha2'].squeeze()
ser_partner_code.name = 'Partner'
dict_codelist['partnerCode'] = ser_partner_code
### Trade Flow Codes:
ser_flow_code = pd.DataFrame(request_session.get(df_cat_reference.loc['flow']['fileuri']).json()['results']).set_index('id').squeeze()
ser_flow_code.name = 'flowCode'
dict_codelist['flowCode'] = ser_flow_code
### Customs Codes:
ser_customs_code = pd.DataFrame(request_session.get(df_cat_reference.loc['customs']['fileuri']).json()['results']).set_index('id').squeeze()
ser_customs_code.name = 'customsCode'
dict_codelist['customsCode'] = ser_customs_code
### Transport Codes:
ser_tansport_code = pd.DataFrame(request_session.get(df_cat_reference.loc['mot']['fileuri']).json()['results']).set_index('id').squeeze()
ser_tansport_code.name = 'motCode'
dict_codelist['motCode'] = ser_tansport_code

In [19]:
### UN COMTRADE: DATA REQUEST PARAMETERS

#str_primary_key = 'e690550ab9414234a6b705220596677a'
str_primary_key = '3f0b8d53d71b401e840d58c90a283176'
str_primary_key = 'd79c218e2e9d464fade1810fd14347d8'
str_goods_type = 'C'
str_services_type = 'S'
str_freq = 'A'
str_goods_class = 'HS'
str_services_class = 'EB'
str_export_flow = 'X'
str_import_flow = 'M'
list_un_reporters = dict_codelist['reporterCode'].index.to_list()
list_un_partners = dict_codelist['partnerCode'].index.to_list()
list_un_goods_ag2 = dict_codelist['clCode'][str_goods_type].index.to_list()
list_un_services_ag2 = dict_codelist['clCode'][str_services_type].index.to_list()
list_periods = str_period = list(map(str, range(date_start.year, date_end.year + 1)))
int_pause_short_sec = 10
int_pause_long_sec = 10
int_period_portion = 5

In [20]:
### UN COMTRADE: DATA REQUEST EXECUTION

def get_un_comtrade_data(str_type, str_freq, str_classification, str_trade_flow, list_reporters, list_partners, list_commodities, list_periods, str_api_key, 
                         int_limit = 99999):
    ### Request preparation:
    str_url_base = 'https://comtradeapi.un.org/data/v1/get/'
    str_url_request = str_url_base + str_type + '/'
    str_url_request = str_url_request + str_freq + '/' 
    str_url_request = str_url_request + str_classification + '?'   
    str_url_request = str_url_request + 'flowCode=' + str_trade_flow 
    str_url_request = str_url_request + '&reporterCode=' + ','.join(list_reporters)
    str_url_request = str_url_request + '&partnerCode=' + ','.join(list_partners)    
    str_url_request = str_url_request + '&cmdCode=' + ','.join(list_commodities)
    str_url_request = str_url_request + '&customsCode=' + 'C00'
    str_url_request = str_url_request + '&motCode=' + '0'    
    str_url_request = str_url_request + '&partner2Code=' + '000'       
    str_url_request = str_url_request + '&period=' + ','.join(list_periods)  
    str_url_request = str_url_request + '&maxrecords=' + str(int_limit)
    ### Request sending:
    bool_loaded = False
    while (not bool_loaded):
        request_session = requests.Session()
        dict_request_headers = {}
        dict_request_headers['Cache-Control'] = 'no-cache'
        dict_request_headers['Ocp-Apim-Subscription-Key'] = str_api_key
        request_session.headers.update(dict_request_headers)
        ### Respond processing:
        print(str_url_request)    
        obj_unc_dataset = request_session.get(str_url_request)
        if not ('count' in obj_unc_dataset.json()):
            print(obj_unc_dataset.json()['error'])
            int_dataset_length = -1
        else:
            int_dataset_length = obj_unc_dataset.json()['count']
        if (int_dataset_length > 0):
            df_dataset_raw = pd.DataFrame(obj_unc_dataset.json()['data'])
            df_dataset_res = df_dataset_raw[['flowCode', 'typeCode', 'period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]
            df_dataset_res.loc[:, 'Flow'] = df_dataset_res['flowCode'].replace(dict_codelist['flowCode']).astype('category').values
            df_dataset_res['Flow'].cat.set_categories(sorted(dict_codelist['flowCode'].values), ordered = True, inplace = True)        
            df_dataset_res.loc[:, 'Type'] = df_dataset_res['typeCode'].replace(dict_codelist['typeCode']).astype('category').values
            df_dataset_res['Type'].cat.set_categories(sorted(dict_codelist['typeCode'].values), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Date'] = (pd.to_datetime(df_dataset_raw['period']) + pd.offsets.BYearEnd()).values
            df_dataset_res.loc[:, 'Reporter'] = df_dataset_res['reporterCode'].astype(str).str.zfill(3).replace(dict_codelist['reporterCode'])\
                                                                                                       .astype('category').values
            df_dataset_res['Reporter'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Partner'] = df_dataset_res['partnerCode'].astype(str).str.zfill(3).replace(dict_codelist['partnerCode']).astype('category').values
            df_dataset_res['Partner'].cat.set_categories(sorted(dict_codelist['partnerCode'].unique()), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Commodity_ID'] = df_dataset_res['cmdCode'].astype('category').values
            df_dataset_res['Commodity_ID'].cat.set_categories(sorted(dict_codelist['clCode']['T'].index), ordered = True, inplace = True)
            df_dataset_res.loc[:, 'Value'] = (df_dataset_res['primaryValue'] / 1000).astype('int32').values
            ### Data clearing:
            df_dataset_res.drop(df_dataset_res[(df_dataset_res['Reporter'] == 'SA') & (df_dataset_res['Partner'] == 'TW')].index, inplace = True)
            df_dataset_res.drop(df_dataset_res[df_dataset_res['Reporter'] == df_dataset_res['Partner']].index, inplace = True)
            df_dataset_res['Value'].clip(lower = 0, inplace = True)
            df_dataset_res = df_dataset_res[['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID', 'Value']]#.dropna()
            print('Loaded Observations Number:', int_dataset_length)
            bool_loaded = True
        elif (int_dataset_length == 0):
            print('Empty Dataset')
            bool_loaded = True            
            df_dataset_res = None            
        else:
            print('Loading Error. Let\'s try once more...')
    return df_dataset_res

In [ ]:
### DATA LOADING ENGINE

list_un_collection = []

for iter_portion in range(-(-len(list_periods) // int_period_portion)):
#for iter_portion in range(-(-len(list_periods) // int_period_portion))[3 : 4]:
    gc.collect()
    list_iter_periods = list_periods[iter_portion *  int_period_portion : (iter_portion + 1) *  int_period_portion]
    ### Goods data loading:
    for iter_comm_id in list_un_goods_ag2:
        print(iter_comm_id, '/', list_iter_periods)        
        ### Export of Goods:
        df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_export_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec) 
        ### Import of Goods:
        df_iter_dataset = get_un_comtrade_data(str_goods_type, str_freq, str_goods_class, str_import_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)                    
    ### Services data loading:
    for iter_comm_id in list_un_services_ag2:      
        print(iter_comm_id, '/', list_iter_periods)        
        ### Export of Services:        
        df_iter_dataset = get_un_comtrade_data(str_services_type, str_freq, str_services_class, str_export_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)                    
        ### Import of Services:        
        df_iter_dataset = get_un_comtrade_data(str_services_type, str_freq, str_services_class, str_import_flow, list_un_reporters, list_un_partners, 
                                               [iter_comm_id], list_iter_periods, str_primary_key)
        if (df_iter_dataset is not None):
            list_un_collection.append(df_iter_dataset)
            time.sleep(int_pause_short_sec)
        else:
            time.sleep(int_pause_long_sec)                        

In [18]:
### DATA CONCATENATION AND SAVING

### Downloaded Data Concatenating:
df_full_dataset = pd.concat(list_un_collection, axis = 0, sort = False, ignore_index = True)
del list_un_collection
ser_full_dataset = df_full_dataset.set_index(['Date', 'Reporter', 'Partner', 'Flow', 'Type', 'Commodity_ID']).squeeze().sort_index()
### Dataset saving:
ser_full_dataset.to_hdf(path_or_buf = str_path_unc_res_all_annual, key = str_key_unc_res, mode = 'w', format = 'table', complevel = 9)

In [ ]:
### EXPORT AND REVERTED IMPORT CONCATENATION

gc.collect()
### File deleting:
if (os.path.exists(str_path_unc_res_flows)):
    os.remove(str_path_unc_res_flows)
### Results container:
list_export_aug = []
### Countries portion length:
int_portion = 5
list_unc_countries = sorted(dict_codelist['partnerCode'].unique())
### Looping over countrie portions:
for iter_num in range(len(list_unc_countries) // int_portion + 1):
#for iter_num in range(2):
    gc.collect()    
    ### Portion of countries selecting:
    list_iter_countries = list(list_unc_countries)[int_portion * iter_num : int_portion * (iter_num + 1)]
    if (len(list_iter_countries) > 0):
        print(list_iter_countries)
        ### Export data loading:
        ser_unc_export = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res,
                                     where = "(Flow = 'Export') & (Reporter in list_iter_countries) & (Partner != 'World')").droplevel('Flow')
        print('Export dataset loaded')
        ### Import data loading:
        ser_unc_import = pd.read_hdf(str_path_unc_res_all_annual, key = str_key_unc_res, 
                                     where = "(Flow = 'Import') & (Partner in list_iter_countries)").droplevel('Flow')
        print('Import dataset loaded')    
        ### Import data reverting:
        ser_unc_import.index.set_names('Partner_Inv', level = 1, inplace = True)
        ser_unc_import.index.set_names('Reporter', level = 2, inplace = True)
        ser_unc_import.index.set_names('Partner', level = 1, inplace = True)
        ser_unc_import = ser_unc_import.swaplevel('Reporter', 'Partner').sort_index()
        print('Import dataset reverted')
        ### Datasets concatenation:
        df_export_aug = pd.concat([ser_unc_export, ser_unc_import], axis = 1, names = 'Source Flow', keys = ['Export', 'Import']).astype('float32')
        del ser_unc_export
        del ser_unc_import    
        gc.collect()    
        print('Export and reverted Import dataset concatenated')
        ### Columns categorization:
        df_export_aug.to_hdf(str_path_unc_res_flows, key = str_key_unc_res, mode = 'a', format = 'table', complevel = 9, append = True)                
        print('Aggregated dataset added to container')
    #    break

In [ ]:
### CIF COEFFICIENTS CALCULATION & IMPLEMENTATION

gc.collect()
### Files deleting:
if (os.path.exists(str_path_export_bilateral)):
    os.remove(str_path_export_bilateral)
if (os.path.exists(str_path_import_bilateral)):
    os.remove(str_path_import_bilateral)
### Getting list of commodities:
#str_date = '1998-12-31'
str_date = '2020-12-31'
list_commodity_id = sorted(pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Date in str_date").index.get_level_values('Commodity_ID').unique())
### Bounds to filter bilateral Import to Export ratio before median calculation:
flo_lower_bound = 1.0
flo_upper_bound = 2.0
### Bilateral median calculation procedure:
def get_obs_median(df_comm):
    ### Export to Import ratio:
    ser_obs_coeff = df_comm['Import'] / df_comm['Export']
    ### Ratio filtering:
    ser_obs_coeff = ser_obs_coeff.loc[(ser_obs_coeff >= flo_lower_bound) & (ser_obs_coeff <= flo_upper_bound)]
    ### Filtered timeseries median as a result:
    return ser_obs_coeff.median()
### Calulation CIF coefficient for all commodities:
for iter_commodity in list_commodity_id:
    gc.collect()
    df_iter_flows = pd.read_hdf(str_path_unc_res_flows, key = str_key_unc_res, where = "Commodity_ID = iter_commodity")
    ser_cif_median = df_iter_flows.droplevel('Commodity_ID').groupby(['Reporter', 'Partner']).apply(get_obs_median)
    ### General commodity median calculation:
    flo_median = ser_cif_median.median()
    print(iter_commodity, ':', flo_median)
    ### Filling missed bilateral values with general commodity median:
    if not (np.isnan(flo_median)):
        ser_cif_median.fillna(flo_median, inplace = True)        
    ser_cif_median.name = 'CIF_Coefficient'              
    ### Adding CIF coefficients to dataset:
    df_export_cif = df_iter_flows.merge(ser_cif_median, left_index = True, right_index = True)
    df_export_cif = df_export_cif.reorder_levels(['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID'])
    ### Import correction:
    df_export_cif['Import_Corrected'] = df_export_cif['Import'] / df_export_cif['CIF_Coefficient']
    ### Export correction:
    df_export_cif['Export_Corrected'] = df_export_cif['Export'] * df_export_cif['CIF_Coefficient']
    ### Combining Export & Import data:
    ser_export_cif = df_export_cif['Export'].combine_first(df_export_cif['Import_Corrected']).astype('float32')
    ser_import_cif = df_export_cif['Import'].combine_first(df_export_cif['Export_Corrected']).astype('float32')
#    del df_export_cif
    gc.collect()
    ser_import_cif = ser_import_cif.reorder_levels(['Date', 'Partner', 'Reporter', 'Type', 'Commodity_ID']).sort_index()                               
    ser_import_cif.index.names = ['Date', 'Reporter', 'Partner', 'Type', 'Commodity_ID']
    gc.collect()
    ### Incorporation options:
    ser_export_cif.squeeze().to_hdf(str_path_export_bilateral, key = str_key_unc_export, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3})
    ser_import_cif.squeeze().to_hdf(str_path_import_bilateral, key = str_key_unc_import, mode = 'a', format = 'table', complevel = 9, append = True, 
                                    min_itemsize = {'Type': 8, 'Commodity_ID': 3}) 
#    break